In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns

In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)
plt.rcParams['axes.grid']=True

In [ ]:
ts_data = pd.read_excel("pma.xlsx")
ts_data = ts_data.reset_index().drop("index", axis=1)
ts_data["time"] = pd.to_datetime(ts_data["time"])
ts_data = ts_data.set_index("time")
ts_data.head()

### Feature Engineering

In [ ]:
seasons = [1, 2, 3, 4]

def map_season(month, day):
    if (month == 12 and day >= 21) or (month == 1) or (month == 2) or (month == 3 and day < 21):
        return seasons[0]  # Winter
    elif (month == 3 and day >= 21) or (month == 4) or (month == 5) or (month == 6 and day < 21):
        return seasons[1]  # Spring
    elif (month == 6 and day >= 21) or (month == 7) or (month == 8) or (month == 9 and day < 21):
        return seasons[2]  # Summer
    else:
        return seasons[3]  # Fall


def is_special_day(date):
    special_days = ['2016-07-06', '2017-06-25', '2016-09-12', '2017-09-01']
    special_days = pd.to_datetime(special_days).date

    return int(date in special_days)


def create_features(df):
    """
    Create time series features based on the time series index.
    """
    df = df.copy()
    df["date"] = df.index.date
    df['hour'] = df.index.hour
    df['dayofweek'] = ((df.index.dayofweek + 8) % 7) + 1
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df["is_special_day"] = df.apply(lambda row: is_special_day(row.date), axis=1)
    df['season'] = df.apply(lambda row: map_season(row.month, row.dayofmonth), axis=1)

    return df

df = create_features(ts_data)

df.head()

### Data cleaning

#### Checking for outliers, missing data and duplicated

In [ ]:
total_missing_values = df.isna().sum().sum()
total_duplicates = df.duplicated().sum()

print(f'total of missing values: {total_missing_values}')
print(f'total of duplicated values: {total_duplicates}')

### EDA

In [ ]:
plt.plot(df.index, df["pma"])

In [ ]:
sns.boxplot(data=df, x='hour', y='pma')

**Analysis**

<mark>TODO</mark>

In [ ]:
sns.boxplot(data=df, x='season', y='pma')

### Trends

#### PMA over all time

Here, we will try to see the development of the maximum power demand over time in 2016 and 2017 by months

In [ ]:
# group
year_month_grouped_pma = df.groupby(['year', 'month']).pma.agg(years_monthly_pma='mean')

title_style = {'family':'serif','color':'darkblue','size':18, 'weight':'bold'}
labels_style = {'family':'serif','color':'black','size':15}
def line_plot(data, x, y, title='', xlabel='', ylabel='', rotate_x=0):
    plt.figure(figsize=[12, 6])
    plt.title(title, fontdict=title_style)
    plt.ylabel(ylabel, fontdict=labels_style)
    plt.xlabel(xlabel, fontdict=labels_style)
    plt.xticks(rotation=rotate_x)
    sns.lineplot(data=data, x=x, y=y)
    plt.show()

# adding a column for the combination year-month
year_month_grouped_pma['year_month'] = year_month_grouped_pma.index.map(lambda x: f'{x[0]}' + '-' + f'{x[1]}')
# year_month_grouped_pma

# plotting the data
line_plot(year_month_grouped_pma, 'year_month', 'years_monthly_pma', 'monthly pma between 2016 and 2017', 'year-month', 'pma', 90)

As we notice, for now, maximum power demand increases mostly during summer from May till October.  
To see this better, let us take the average pma between the two years for ech month:

#### PMA over months

In [ ]:
monthly_grouped_pma = year_month_grouped_pma.groupby('month').years_monthly_pma.agg(average_monthly_pma='mean')
# monthly_grouped_pma
line_plot(monthly_grouped_pma, 'month', 'average_monthly_pma', 'Average Monthly pma', 'months', 'pma')

In [ ]:
sns.boxplot(data=df, x='month', y='pma')

Great, It is clear enough now.

#### PMA over days

Now, let us find the hours of the day with most power demand.

In [ ]:
hourly_grouped_pma = df.groupby('hour').pma.agg(hourly_pma='mean')
#hourly_grouped_pma
line_plot(hourly_grouped_pma, 'hour', 'hourly_pma', 'Average Hourly PMA During One Day', 'Hour', 'pma')

In [ ]:
sns.boxplot(data=df, x='hour', y='pma')

<mark>TODO</mark>
- Add analysis

However, this one is for both two years, let us do it for each season and try to compare.

In [ ]:
seasoned_hourly_grouped_pma = df.groupby(['season', 'hour']).pma.agg(seasoned_hourly_pma='mean')
fall_hourly = seasoned_hourly_grouped_pma[seasoned_hourly_grouped_pma.index.get_level_values('season')=='Fall']
winter_hourly = seasoned_hourly_grouped_pma[seasoned_hourly_grouped_pma.index.get_level_values('season')=='Winter']
spring_hourly = seasoned_hourly_grouped_pma[seasoned_hourly_grouped_pma.index.get_level_values('season')=='Spring']
summer_hourly = seasoned_hourly_grouped_pma[seasoned_hourly_grouped_pma.index.get_level_values('season')=='Summer']

plt.figure(figsize=[14, 6])
plt.title('Average Hourly PMA During Each Season')
plt.ylabel('seasoned pma')
plt.xlabel('hour')

season_num_to_name = {
    1: "Winter",
    2: "Spring",
    3: "Summer",
    4: "Fall"
}

for season in seasons:
    seasoned_hourly = seasoned_hourly_grouped_pma[seasoned_hourly_grouped_pma.index.get_level_values('season')==season]
    #line_plot(seasoned_hourly, 'hour', 'seasoned_hourly_pma', f'Average Hourly Demand in {season}', 'Hour', 'pma')
    sns.lineplot(data=seasoned_hourly, x='hour', y='seasoned_hourly_pma', label=season_num_to_name[season])

plt.show()

#### PMA in Week Days

We'll now discover which day in the week has the highest consumption of energy

In [ ]:
# We plot the average usage of each day of the week
dayName_grouped_pma = df.groupby('dayofweek').pma.agg(average_pma='mean')

# We sort the days of the week
dayName_grouped_pma = dayName_grouped_pma.reindex(range(8))

# We plot the data
line_plot(dayName_grouped_pma, 'dayofweek', 'average_pma', 'Average pma During Each Day of the Week', 'Day of the Week', 'pma')


> We notice that the consumption is low in the weekends because most of people rest or go for picnics

#### PMA in special days (Eids Days)

Let us discover how was the usage of people in "Eids", we might find some high usage due to family gathering



In [ ]:
# We first define the special days (Eids in 2016 and 2017)
special_days_df = df.loc[df["is_special_day"] == 1]
special_days_df

Let's compare the usage in Eid days to the overall usage

In [ ]:
# We plot a comparison between the average hourly pma in the special days and the average hourly pma in the normal days
normal_days_df = df[df["is_special_day"] == 0]
normal_days_hourly_grouped_pma = normal_days_df.groupby('hour').pma.agg(normal_hourly_pma='mean')
special_days_hourly_grouped_pma = special_days_df.groupby('hour').pma.agg(special_hourly_pma='mean')
# we plot the comparison
plt.figure(figsize=[12, 6])
plt.title('Average Hourly PMA During Special Days and Normal Days', fontdict=title_style)
plt.ylabel('pma', fontdict=labels_style)
plt.xlabel('hour', fontdict=labels_style)
sns.lineplot(data=normal_days_hourly_grouped_pma, x='hour', y='normal_hourly_pma', label='normal days')
sns.lineplot(data=special_days_hourly_grouped_pma, x='hour', y='special_hourly_pma', label='special days')
plt.show()

sns.boxplot(data=df, x='is_special_day', y='pma')

> It's remarkable that the usage in Eids is higher than in the normal days due to family gathering and especially that Eids were in the summer

#### Key notes

* Power demand reaches its maximum values during summer. Which logical, most people are in holidays thus staying at home most of the time compared to the rest of the year.
* During one day, PMA reaches its climax at around 8pm.
* During Winter, max demand is at its peak before 8pm, and after that time it starts decreasing. This could be due to many reasons. One of them is that people tend to sleep earlier at winter. Meanwhile during summer, it reaches its climax after 8pm and higher values as well 1pm and 4pm. Because, most people are at their homes with their AC (Air Conditioner) on at those times due to high temperatures outside.
* Demand is low in all seasons during night (most logically) and medium during day where everyone are doing their activities and daily tasks.
* Demand is low in weekends which is also logic because people are resting and companies are off
* Demand is higher in Eids because of family gathering which cause a high consumption of electricity (lights on everywhere, hair dryer for women..etc)

### Data preprocessing

#### Grouping by days
- Our data has each hour in a row, so we need to take the median of each 24 hours to represent that day
- We choose the median to avoid outliers' effect.

In [ ]:
df_daily = df.groupby("date").agg({"pma": "median", "tmp": "median", "year": "min", "month": "min", "dayofmonth": "min", "dayofyear": "min", "is_special_day": "min", "season": "min", "dayofweek": "min", "quarter": "min"})

df_daily.index = pd.to_datetime(df_daily.index)

df_daily.head()

#### Cyclic attribute representation
Since our data is a time series, we have many attributes that are cyclic, for example: month, which starts with 0 and ends with 11. We all know that month 11 (December) and month 1 (January) are right next to each other, but with this representation 11 is way bigger than 1, so if we apply any clustering or regression algorithm it will think that they are very appart from each other and fail to determine the real patterns in the data.

In [ ]:
def cyclical_transormation(df, columns: list, remove_original=False):
    df = df.copy()
    for col in columns:
        max_col_val = df[col].max()
        df[f"{col}_sin"] = np.sin((2*np.pi*df[col]) / max_col_val)
        df[f"{col}_cos"] = np.cos((2*np.pi*df[col]) / max_col_val)

    if remove_original:
        df = df.drop(columns=columns)

    return df

In [ ]:
cyclical_columns = ["year", "month", "dayofmonth", "dayofyear", "season", "dayofweek", "quarter"]
df_cyclic = cyclical_transormation(df_daily, columns=cyclical_columns, remove_original=True)
df_cyclic.head()

## Modeling

### PMA Regression to find most important features in order to justify the assumptions above

In [ ]:
def train_test_split(df, day_threshold=4):
    train = df.loc[(df.reset_index().index + 1) % day_threshold != 0]
    test = df.loc[(df.reset_index().index + 1) % day_threshold == 0]

    return train, test

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

cols = df_cyclic.columns
FEATURES = cols[cols != "pma"]
TARGET = 'pma'

train, test = train_test_split(df_cyclic)

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

# reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',
#                        n_estimators=5000,
#                        early_stopping_rounds=50,
#                        objective='reg:linear',
#                        max_depth=5,
#                        learning_rate=0.01)

gsc = RandomizedSearchCV(
            estimator=xgb.XGBRegressor(),
            param_distributions={
                        "learning_rate": [0.005, 0.01, 0.02, 0.05, 0.1],
                        "max_depth": [ 3, 4, 5, 6, 8],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "n_estimators": [2000, 3000, 5000, 10000],
                        "colsample_bytree":[ 0.3, 0.4],
                        },
            cv=3, scoring='neg_mean_squared_error', verbose=100, n_jobs=-1)

grid_result = gsc.fit(X_train, y_train)


In [ ]:
print('\n All results:')
print(grid_result.cv_results_)
print('\n Best estimator:')
print(grid_result.best_estimator_)
print('\n Best hyperparameters:')
print(grid_result.best_params_)

In [ ]:

reg = xgb.XGBRegressor(
    n_estimators=grid_result.best_params_["n_estimators"],
    max_depth=grid_result.best_params_["max_depth"],
    colsample_bytree=grid_result.best_params_["colsample_bytree"],
    min_child_weight=grid_result.best_params_["min_child_weight"],
    learning_rate=grid_result.best_params_["learning_rate"]
)


In [ ]:

reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=100)

In [ ]:
fi = pd.DataFrame(data=reg.feature_importances_,
             index=reg.feature_names_in_,
             columns=['importance'])
fi.sort_values('importance').plot(kind='barh', title='Feature Importance')
plt.show()

##### Key Notes
- we see that Quarter, Season and temperature are one of the most important features

In [ ]:
test['prediction'] = reg.predict(X_test)
df_pred = df_daily.merge(test[['prediction']], how='left', left_index=True, right_index=True)


In [ ]:

plt.plot(df_pred["pma"])
plt.plot(test["prediction"])
plt.legend(['Truth Data', 'Predictions'])
plt.title('Raw Data and Predictions')
plt.show()

In [ ]:
mse_score = mean_squared_error(test['pma'], test['prediction'])
rmse_score = np.sqrt(mse_score)

print(f'MSE Score on Test set: {mse_score:0.2f}')
print(f'RMSE Score on Test set: {rmse_score:0.2f}')

#### Key notes
- our regression model succeeds quiet well on predicting the values of the test data
- Using our model we can predict the PMA values of future records

### Clustering

#### K-Means clustering

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score

def pca_transform(df, explained_variance_threshold=0.95):
    pca = PCA(n_components=explained_variance_threshold)
    
    return pca.fit_transform(df)

def normalise(df):
    standard_scaler = StandardScaler()
    
    return standard_scaler.fit_transform(df)

In [ ]:
train, test = train_test_split(df_cyclic)

In [ ]:

# we remove "pma" since we probably don't know it in the future
# and so that our model doesn't depend on it in that case

# also not sure if we really need to split data in clustering or not
train_no_pma = train.drop(columns=["pma"])
test_no_pma = test.drop(columns=["pma"])

# when we use PCA, we get inconsistent clustering (example: putting december 31st and january 1st in diff clusters), so ig we won't use it
reduced_train = normalise(pca_transform(train_no_pma, explained_variance_threshold=0.95))
reduced_test = normalise(pca_transform(test_no_pma, explained_variance_threshold=0.95))



In [ ]:
def k_means_plot_silhouette_score(df, max_k=11):
    silhouette = []
    k_value = range(2, 11) # silhouette needs at least 2 clusters

    for k in k_value:
        kmeans = KMeans(n_clusters=k, n_init=10)
        kmeans.fit(df)
        cluster_labels = kmeans.labels_
        silhouette.append(silhouette_score(df, cluster_labels))

    plt.figure(figsize=[10, 6])
    plt.plot(k_value, silhouette, 'bx-')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Silhouette')
    plt.title('Silhouette Metric To Find Optimal Number Of Clusters')
    plt.show()

    return k_value[np.array(silhouette).argmax(axis=0)]

def perform_kmeans_clustering(df, k):
    kmeans = KMeans(n_clusters=k, n_init=10)
    cluster = kmeans.fit(df)

    return cluster

In [ ]:
def pipeline(df, supposed_target="pma", k_override=None):
    no_target_df = df.drop(columns=[supposed_target])
    k_value = k_means_plot_silhouette_score(no_target_df) if k_override is None else k_override
    cluster = perform_kmeans_clustering(no_target_df, k=k_value)
    plt.scatter(df.index, df["pma"], c=cluster.labels_)
    print(f'SSE = {cluster.inertia_}')

In [ ]:
pipeline(df_cyclic, k_override=3)